In [1]:
import leafmap
import solara
import pystac_client
import planetary_computer
import odc.stac
import geopandas as gpd
import dask.distributed
import matplotlib.pyplot as plt
import rioxarray
from osgeo import gdal


In [2]:

# Stashed public copies of NPS polygons and CalFire polygons


zoom = solara.reactive(14)
center = solara.reactive((34, -116))
nps = gpd.read_file("/vsicurl/https://huggingface.co/datasets/cboettig/biodiversity/resolve/main/data/NPS.gdb")
calfire = gpd.read_file("/vsicurl/https://huggingface.co/datasets/cboettig/biodiversity/resolve/main/data/fire22_1.gdb",  layer = "firep22_1")


In [3]:



# fire = gpd.read_file("/vsizip/vsicurl/https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/MTBS_Fire/data/composite_data/burned_area_extent_shapefile/mtbs_perimeter_data.zip"

# extract and reproject the Joshua Tree NP Polygon
jtree = nps[nps.PARKNAME == "Joshua Tree"].to_crs(calfire.crs)
# All Fires in the DB that intersect the Park
jtree_fires = jtree.overlay(calfire, how="intersection")

# Extract a polygon if interest.   > 2015 for Sentinel, otherwise we can use LandSat
recent = jtree_fires[jtree_fires.YEAR_ > "2015"]
big = recent[recent.Shape_Area == recent.Shape_Area.max()].to_crs("EPSG:4326")
box = big.buffer(0.01).bounds.to_numpy()[0]  # Fire bbox + buffer
#box = jtree.to_crs("EPSG:4326").bounds.to_numpy()[0] # Park bbox


In [4]:
from datetime import datetime, timedelta
alarm_date = datetime.strptime(big.ALARM_DATE.item(), "%Y-%m-%dT%H:%M:%S+00:00")  
before_date = alarm_date - timedelta(days=14)
after_date = alarm_date + timedelta(days=14)

In [5]:
search_dates = big.ALARM_DATE.item() + "/" + big.CONT_DATE.item()
search_dates = before_date.strftime("%Y-%m-%d") + "/" + after_date.strftime("%Y-%m-%d")

In [6]:

def stac_search(box, datetime):    
    # STAC Search for this imagery in space/time window
    items = (
        pystac_client.Client.
        open("https://planetarycomputer.microsoft.com/api/stac/v1",
            modifier=planetary_computer.sign_inplace).
        search(collections=["sentinel-2-l2a"],
                bbox=box,
                datetime=datetime,
                query={"eo:cloud_cover": {"lt": 10}}).
        item_collection())
    return items

def compute_nbs(items, box):
    # Time to compute:
    client = dask.distributed.Client()
    # landsat_bands = ["nir08",  "swir16"]
    sentinel_bands = ["B08", "B12", "SCL"] # NIR, SWIR, and Cloud Mask

    # The magic of gdalwarper. Can also resample, reproject, and aggregate on the fly
    data = odc.stac.load(items,
                        bands=sentinel_bands,
                        bbox=box
    )
    # Compute the Normalized Burn Ratio, must be float
    swir = data["B12"].astype("float")
    nir = data["B08"].astype("float")
    # can resample and aggregate in xarray. compute with dask
    nbs = (((nir - swir) / (nir + swir)).
        #  resample(time="MS").
        #  median("time", keep_attrs=True).
            compute()
    )
    return nbs

items = stac_search(box, search_dates)
nbs = compute_nbs(items, box)



In [7]:

nbs.isel(time=0).rio.to_raster(raster_path="before.tif", driver="COG")
nbs.isel(time=(nbs.time.size-1)).rio.to_raster(raster_path="after.tif", driver="COG")


In [8]:

before_url = "https://huggingface.co/datasets/cboettig/solara-data/resolve/main/before.tif"
after_url = "https://huggingface.co/datasets/cboettig/solara-data/resolve/main/after.tif"


In [9]:
class Map(leafmap.Map):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Add what you want below
        # self.add_gdf(jtree, layer_name = "Joshua Tree NP")
       # self.add_gdf(jtree_fires)
        self.add_gdf(big, later_name = big.FIRE_NAME.item())
        #self.add_raster("before.tif", layer_name = "before", colormap="viridis")
        #self.add_raster("after.tif", layer_name = "after", colormap="viridis")
        self.split_map(before_url, after_url)
        #self.add_stac_gui()


@solara.component
def Page():
    with solara.Column(style={"min-width": "500px"}):
        # solara components support reactive variables
        # solara.SliderInt(label="Zoom level", value=zoom, min=1, max=20)
        # using 3rd party widget library require wiring up the events manually
        # using zoom.value and zoom.set
        Map.element(  # type: ignore
            zoom=zoom.value,
            on_zoom=zoom.set,
            center=center.value,
            on_center=center.set,
            scroll_wheel_zoom=True,
            toolbar_ctrl=False,
            data_ctrl=False,
            height="780px",
        )
        solara.Text(f"Zoom: {zoom.value}")
        solara.Text(f"Center: {center.value}")


In [10]:
Page()

Cannot show ipywidgets in text